In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('precision', 4)
np.set_printoptions(precision=3)

import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-paper')

## 목차
1. Read the dataset (after extracting)
2. Preprocessing
3. EDA
4. Modeling
5. Evaluation

## Read the dataset
- 데이터셋: 유저별(iduser) 문서 사용행동에 대한 횟수와 그룹 특성(group)를 처리한 데이터
    - 3개의 테이블 소스에서 Raw 데이터를 가공: ``` groupby("iduesr").agg(count,,,sum,,,etc)``` 이후에 조인함
    - 결제 타이밍 기준(유저마다 다름)으로 이전 30일의 행동 기준으로 데이터 추출
- 간혹 csv를 불러올 때 unnamed 라는 컬럼이 자동으로 생성되므로, index_col=0 이라는 명령어를 통해 처리

In [ ]:
df = pd.read_csv("testset.csv", index_col=0)

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
df.shape

### Column Info.
- ✭iduser: 식별값
- mdutype: 중요x
- ✭✭group: y, 결제(mdu) vs 비결제(sdu) 정보
- ✭viewCount: 보기 횟수
- ✭editCount: 편집 횟수
- ✭shareCount: 공유 횟수
- ✭searchCount: 검색 횟수
- ✭coworkCount: 공동작업 횟수
- add: 파일 추가 횟수
- del: 파일 삭제 횟수
- move: 파일 이동 횟수
- rename: 파일명 변경 횟수
- adddir: 폴더 생성
- movedir: 폴더 이동
- ✭✭visdays: 방문일수
- ✭openCount: 열기 횟수
- ✭saveCount: 저장 횟수
- ✭exportCount: 내보내기 횟수
- viewTraffic: 보기 용량(파일 사이즈)
- editTraffic: 편집 용량
- exportTraffic: 내보내기 용량
- traffic: 전체 용량

![img](po_detail.png)


### Pandas DF index 지정
- 유저ID와 같은 유니크 값(primary key)를 인덱스로 지정하는 것이 편리함 (pandas 장점)
- 그렇지 않으면, 추후 scaling 이나 모델 학습 등을 할때 매번 슬라이싱으로 처리해야함 

In [ ]:
df.set_index("iduser", inplace=True)

In [ ]:
df.head()

In [ ]:
df.drop("mdutype", axis=1, inplace=True)

In [ ]:
# check missing values in each cols
df.isnull().sum().plot(kind='barh', color='darkblue', figsize=(10,6))

plt.title("Missing value counts")
plt.grid(color='lightgrey', alpha=0.5, linestyle='--')
plt.tight_layout()

### 결측치 처리

- 추후에는 협의를 통해 결측치 발새 이유 파악 및 예방에 노력을 기울이는 것이 필요
- 결측치 처리 방법
    - 가장 쉬운 방법은 Null이 포함 행을 모두 제거하는 것이다
    - 사례(observation)이 많다면 이 방법을 사용하는 것이 가능하다
    - 평균, 중앙치, 최빈치, 간단한 예측 모델활용해서 imputation

    - 만약 샘플수가 충분하지 않을 경우, Pandas의 fillna() 명령어로 Null 값을 채우는 것이 가능하다. 
    - 연속형인 경우 Mean이나 Median을 이용하고 명목형인 경우 Mode(최빈치)나 classification 모델을 통해 Null 값을 대체할 수 있다.

```python
# Null 값을 median, mean으로 대체하는 코드 예제
df.fillna(df.med())
df.fillna(df.mean()) 

# Scikit-learn Imputation을 이용하여 명목변수의 Null 값을 Mode로 대체한 예제
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values = 'NaN', strategy='most_frequent', axis=0)
df['X'] = imp.fit_transform(df['X'])

# 참고: http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Imputer.html
```

In [ ]:
# 우선 group 컬럼이 null 경우만 선택
df[df['group'].isnull() == True].head(10)

In [ ]:
# visdays 컬럼이 null 경우만 선택
df[df['visdays'].isnull() == True].head(10)

In [ ]:
# 특정 열을 기준으로 dropna
df1 = df.dropna(subset=['visdays'])

In [ ]:
df1.head(10)

In [ ]:
df1.isnull().sum()

#### Q) visiday, group 을 제외한 나머지 항목이 nan이면 drop 해볼까요?

In [ ]:
# subset 용 컬럼 설정
mycols = df1.columns[1:].drop('visdays')

# mycols 컬럼들을 기준으로 drop
df2 = df1.dropna(subset=mycols, how='all')

In [ ]:
df2.head(10)

In [ ]:
df2.isnull().sum()

In [ ]:
df2[df2['viewCount'].isnull() == True][:10]

In [ ]:
df2[df2['add'].isnull() == True][:10]

In [ ]:
df2[df2['openCount'].isnull() == True][:10]

In [ ]:
df2[df2['traffic'].isnull() == True][:10]

#### add, del ~ movedir 까지는 파일 management 관련 변수이므로 상대적 중요도 낮음
- 변수 삭제를 고려해볼 수 있으나 우선 pass
- 우선 메꾸고 추후 제거도 고려 가능

In [ ]:
# zero로 imputaion
df2 = df2.fillna(0)

In [ ]:
df2.info()

In [ ]:
df2.head()

In [ ]:
# 간략한 기술 통계 확인
df2.describe()

### Missing Value 처리 가이드

- Missing Value 파악을 위해 `df.info()` 가장 처음에 이용
- 만약 np.nan으로 적절히 missing value로 불러왔다면 info() 이용 가능하지만,
- '', ' ' 이런식의 공백이나 다른 방식(.)으로 처리되어 있다면, 모두 repalce 처리해야함
- `df.info()`를 실행했을 때, 누가봐도 float or int 인데 object(string)으로 되어 있다면 이런 사레가 포함될 가능성 높음

In [ ]:
# 가짜 dataframe 생성
tt = df2[['group', 'viewCount']]

tt.head()

In [ ]:
tt.info()

In [ ]:
# np.nan 대신 ''로 수집된 경우
tt.loc[10100037810674,'viewCount'] = ''
tt.loc[10100036273719,'viewCount'] = ''

# np.nan 대신 '.'로 수집된 경우
tt.loc[10100034746743,'group'] = '. '
tt.loc[10100016781863,'group'] = '. '

# if missing is zero.......?

In [ ]:
tt.head(10)

In [ ]:
tt.info() # if continous var such as viewcount is object object?

In [ ]:
# 만약 큰 데이터셋에서는 찾는 경우는 정렬을 이용
tt.sort_values("viewCount", ascending=False).head(10)

In [ ]:
tt.query("viewCount == ''")

In [ ]:
tt.sort_values("group", ascending=True).head(10)

In [ ]:
tt.query("group == '.'")

In [ ]:
# 공백 제거
tt['group'] = tt['group'].str.strip()

In [ ]:
tt.query("group == '.'")

In [ ]:
# 만약 변수에 공백이 있을 경우
tt.columns = ['group ', ' viewCount']

In [ ]:
tt.head()

In [ ]:
# strip 함수 이용
tt.columns = tt.columns.str.strip()

In [ ]:
# if no float, replace with np.nan
for i in tt.index:
    if type(tt.loc[i, 'viewCount']) == float:
        tt.loc[i, 'viewCount'] == tt.loc[i, 'viewCount']    
    else:
        tt.loc[i, 'viewCount'] = np.nan

In [ ]:
tt.head()

In [ ]:
# replace 이용 방식 (단 에러 값을 정확히 알고 있어야 함)
tt['group'] = tt['group'].replace('.', np.nan)

In [ ]:
tt.head(10)

In [ ]:
tt.isnull().sum()

In [ ]:
# fill null with mean
tt['viewCount'] = tt['viewCount'].fillna(tt.viewCount.mean())

# fill null with mode
tt['group'] = tt['group'].fillna(tt.group.value_counts().index[0])

In [ ]:
tt.head(10)

### 결측치를 처리할 때 고려할 점
- 결측치를 처리할 경우에도 도메인 지식은 유용하게 사용된다.
- 인적, 기계적 원인임이 판명되면, 협업자와 지속적으로 노력해 결측치를 **사전에 발생하지 않도록 조치**하는 것이 좋다
- 수치형인 경우 의미상으로 0으로 메꾸는 것이 맞는지 아니면 평균이나 중앙치, 최빈치가 맞는지 정확히 판단해야 한다.
    - 예를 들어 viewCount가 1이상인데, edit, export가 missing인 경우 (도메인 지식을 통해) 0으로 메꾸는 것이 가능하다.
    - 왜냐하면, ViewCount가 다른 행동에 선행하는 개념이기 때문에 위와 같은 의사결정이 가능하다
- 특히 **숫자 0과 null 과 같은 결측치는 완전히 다른 개념**이니 유의해야 한다.
    - 0: -1과 1 사이의 가운데 값(숫자)임. '제로'라는 의미를 지니고 있음.
    - null or nan: 미지의 값 (모름)
- 만약 y label(위 샘플 데이터에서는 'group')에 결측치가 있다면 그냥 drop\
- pandas 결측치 관련 API: https://pandas.pydata.org/pandas-docs/stable/missing_data.html
- 참고 블로그: https://machinelearningmastery.com/handle-missing-data-python/

### 이상치(Outlier) 처리 방법

#### 0. 이상치란?

>In statistics, an outlier is a data point that differs greatly from other values in a data set. Outliers are important to keep in mind when looking at pools of data because they can sometimes affect how the data is perceived on the whole.


#### 1. 표준편차(standard deviation) 이용
- 현재 분포에서 표준편차 기준 +3 이상이거나 -3 이하인 경우 극단치로 처리
- 정규분포일 경우 유용

#### 2. 표준점수(z-score) 이용
- 평균을 0, 표준편차를 1로 맞춘후 표준편차 이용 (+-3 벗어난 경우 이상치로 판정)
- 정규분포일 경우 유용

![img](out_std.png)


#### 3. IQR 방식
- 75 percentile + IQR * 1.5 이상이거나 25 percentile + IQR * 1.5 이하인 경우 극단치로 처리
- 정규분포 아닐 경우 robust한 편

![img](out_iqr.png)

#### 4. 기타 방법
- Rule-based (ex, percentile 95% 이상이면 제거)
- Binning (연속변인을 카테고리형으로 변환)

#### 5. 발생이유
- 입력 오류, 측정 오류, 고의성, 처리시 에러, 샘플링에러, 자연발생 등
- 예방이 가장 좋음

In [ ]:
# 이상치 처리 대상 데이터 (NA 제거한 dataframe)
df2.head()

In [ ]:
# 숫자형 변수만 선택
# df3 = df2.drop("group", axis=1) 
df3 = df2._get_numeric_data() 

In [ ]:
df3.describe()

In [ ]:
fig, ax = plt.subplots(2, 2)

df3['viewCount'].plot(kind='box', ax=ax[0, 0], figsize=(5, 6));
df3['editCount'].plot(kind='box', ax=ax[0, 1], figsize=(5, 6));
df3['shareCount'].plot(kind='box', ax=ax[1, 0], figsize=(5, 6));
df3['searchCount'].plot(kind='box', ax=ax[1, 1], figsize=(5, 6));

plt.title("Boxplot")
plt.tight_layout()

In [ ]:
df3.plot(kind='scatter', x='viewCount', y='editCount');

In [ ]:
# 표준편차 이용 (6-sigma): 각 컬럼별 표준편차에서 +- std 벗어난 경우 제외
def std_based_outlier(df):

    for i in range(0, len(df.iloc[1])): 
        df = df[~(np.abs(df.iloc[:,i] - df.iloc[:,i].mean()) > (3*df.iloc[:,i].std()))]

        return(df)

In [ ]:
df3_std = std_based_outlier(df3)

In [ ]:
df3_std.describe()

In [ ]:
df3_std.plot(kind='scatter', x='viewCount', y='editCount');

In [ ]:
## 가장 자주 쓰이는 방식
# z-score 이용: 표준점수로 변환후 +-3 std 벗어나는 경우 제거
from scipy import stats

df3_zscore = df3[(np.abs(stats.zscore(df3)) < 3).all(axis=1)]

In [ ]:
df3_zscore.describe()

In [ ]:
fig, ax = plt.subplots(2, 2)

df3_zscore['viewCount'].plot(kind='box', ax=ax[0, 0], figsize=(5, 6));
df3_zscore['editCount'].plot(kind='box', ax=ax[0, 1], figsize=(5, 6));
df3_zscore['add'].plot(kind='box', ax=ax[1, 0], figsize=(5, 6));
df3_zscore['searchCount'].plot(kind='box', ax=ax[1, 1], figsize=(5, 6));

plt.tight_layout()

In [ ]:
df3_zscore.plot(kind='scatter', x='viewCount', y='editCount');

#### original std vs z-score std
- 원 데이터로 처리시 매우 보수적인 결과 -> 효과가 낮아서 자주 쓰이지 않음
- 일반적으로 z-score 자주 활용, 단 정규분포에 효과적
- 정규분포가 아닐 경우, IQR 고려

In [ ]:
# IQR
df3 = df2._get_numeric_data() 

for i in range(0, len(df3.iloc[1])): 
        
    q1 = df3.iloc[:,i].quantile(0.25)
    q3 = df3.iloc[:,i].quantile(0.75)
    iqr = q3-q1
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr       

    df3 = df3[(df3.iloc[:,i] >= fence_low) & (df3.iloc[:,i] <= fence_high)]

In [ ]:
df3.describe()

#### 최종 검토
- IQR: 분포가 쏠려 있어서 (거의 median이 0), 조금만 벗어나도 이상치로 판정
- 최종 선정은 zscore 기준으로 우선 선택!!

In [ ]:
df3_zscore.describe()

In [ ]:
# max가 0인 컬럼은 제거
cols_max = df3_zscore.describe().loc['max']
drop_cols = cols_max[cols_max == 0]

df3_zscore.drop(drop_cols.index, axis=1, inplace=True)

In [ ]:
df3_zscore.describe()

In [ ]:
fig, ax = plt.subplots(2, 2)

df3_zscore['viewCount'].plot(kind='box', ax=ax[0, 0], figsize=(5, 6));
df3_zscore['editCount'].plot(kind='box', ax=ax[0, 1], figsize=(5, 6));
df3_zscore['add'].plot(kind='box', ax=ax[1, 0], figsize=(5, 6));
df3_zscore['searchCount'].plot(kind='box', ax=ax[1, 1], figsize=(5, 6));

plt.tight_layout()

In [ ]:
df3_zscore.plot(kind='scatter', x='viewCount', y='viewTraffic');

### 분포 변환
- Transformation
    - if right skewed: Log, Sqrt, cube root functions
    - if left skwed: square
- left_distribution: X^3
- mild_left: X^2
- mild_right: sqrt(X)
- right: ln(X)
- servere right: 1/X    
    
![img](skwed.png)

- source: http://seismo.berkeley.edu/~kirchner/eps_120/Toolkits/Toolkit_03.pdf

- Reference
    - http://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing
    - http://scikit-learn.org/stable/modules/preprocessing.html

In [ ]:
df4 = df3_zscore.copy()

In [ ]:
df4.head()

In [ ]:
df4.info()

In [ ]:
# 분포를 간단히 확인 with IQR, MIN, MAX, MEAN, STD
df4.describe()

In [ ]:
# Outlier 미처리시 skewness 더 심해짐
df4.hist(bins=15, color='darkblue', figsize=(18,14), grid=False); 

plt.grid(False)

In [ ]:
# log 함수 적용 (if right skewed)
df4_log = df4.apply(lambda x: np.log(x+1))    

df4_log.describe()

In [ ]:
df4_log.hist(bins=15, color='brown', figsize=(18,14), grid=False);

In [ ]:
# 다른 함수 적용
df4_cube_root = df4.apply(lambda x: x ** (1. / 3))    

df4_cube_root.describe()

In [ ]:
df4_cube_root.hist(bins=15, color='brown', figsize=(18,14), grid=False);

#### Q) 다양한 여러 함수를 적용해보시고 가장 정규분포로 잘 변환되는 함수를 찾아주세요.

In [ ]:
# 다른 함수 적용

#### 우선 분포 변환은 Pass
- 원 분포대로 모델링 구축하고 추후 개선시 transform 진행

### 단위 표준화 (Scaling)
- 모든 변수의 단위를 동일한 기준(스케일)로 통일
    - 이번 사례의 경우 tarffic(byte) 변수로 인해, 필수적인 과정
- Standard Scaler (Mean: 0, std: 1)
- MinMax Scaler (default: min=0, max=1)
- Robust Scaler (x - q1 / q3-q1)
- Source: http://benalexkeen.com/feature-scaling-with-scikit-learn/

![img](scale_std.png)
![img](scale_minmax.png)
![img](scale_robust.png)

In [ ]:
df4_std_scale = df4.copy() # after removes outliers

In [ ]:
df4_std_scale.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

for c in df4_std_scale:
    df4_std_scale[c] = StandardScaler().fit_transform(np.array(df4_std_scale[c]).reshape(-1, 1)).round(4)

In [ ]:
df4_std_scale.describe().round(2)

#### Q) 단위 변화후 분포에는 어떠한 변화가 있을까요?

In [ ]:
df4.hist(bins=15, color='darkblue', figsize=(18,14), grid=False);

In [ ]:
df4_std_scale.hist(bins=15, color='brown', figsize=(18,14), grid=False);

In [ ]:
df4_minmax_scale = df4.copy()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

for c in df4_minmax_scale:
    df4_minmax_scale[c] = MinMaxScaler().fit_transform(np.array(df4_minmax_scale[c]).reshape(-1,1).round(4))

In [ ]:
df4_minmax_scale.describe()

In [ ]:
df4_robust_scale = df4.copy()

In [ ]:
from sklearn.preprocessing import RobustScaler

for c in df4_robust_scale:
    df4_robust_scale[c] = RobustScaler().fit_transform(np.array(df4_robust_scale[c]).reshape(-1,1).round(4))

In [ ]:
df4_robust_scale.describe()

In [ ]:
# dataset after removes outliers, not transform, no scale
df4.head()

In [ ]:
df5 = df4.join(df2['group'])

In [ ]:
df5.head()

In [ ]:
df5.describe() 

#### 다중공선성 issues
- 변수 삭제 1): EX) editTraffic, exportTraffic, viewTraffic, openCount
- 변수 삭제 2): 분포가 정상이 아닌 경우 (우선 유지)
- 변수 축소 EX) PCA, Factor Analysis, etc

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))

sns.heatmap(df5.corr(), annot=True, annot_kws={"size": 13}, cmap='Purples');

In [ ]:
# 상관관계 높은 변수 제거
drop_cols = ['editTraffic', 'exportTraffic', 'viewTraffic', 'openCount']

In [ ]:
df5.drop(drop_cols, axis=1, inplace=True)

In [ ]:
df5.head()

#### Imbalance Issues
- 결제자 여부에 대한 데이터 사례가 불충분 => 모델이 sdu 로 대부분 예측하는 결과
    - SDU(0)를 SDU(0)로 예측하는 정확도(True Negative)는 높을 수있으나, MDU(1)를 MDU(1) 예측하는 TP는 낮을 것으로 예상
- how to handle imbalance    
    - https://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/

In [ ]:
df5.group.value_counts()

In [ ]:
df5.group.value_counts().plot(kind='bar', color='darkblue', rot=0)

In [ ]:
df5['group'] = np.where(df5['group'] == 'sdu', 0, 1) 

In [ ]:
df5.head()

### 분류 모델 종류
 - **Logistic Regression**
     - Logistic regression fits a logistic model to data and makes predictions about the probability of an event (between 0 and 
 - **Naive Bayes**
    - Naive Bayes uses Bayes Theorem to model the conditional relationship of each attribute to the class variable
 - **k-Nearest Neighbor**
    - The k-Nearest Neighbor (kNN) method makes predictions by locating similar cases to a given data instance (using a similarity function) and returning the average or majority of the most similar data instances. The kNN algorithm can be used for classification or regression.
 - **Trees-based model**
    - Classification and Regression Trees (CART) are constructed from a dataset by making splits that best separate the data for the classes or predictions being made. The CART algorithm can be used for classification or regression
 - **Random Forest**
    - Random Forest is a machine learning algorithm used for classification, regression, and feature selection. It's an ensemble technique, meaning it combines the output of one weaker technique in order to get a stronger result. The weaker technique in this case is a decision tree. Decision trees work by splitting the and re-splitting the data by features. If a decision tree is split along good features, it can give a decent predictive output    
 - **SVM (Support Vector Machines)**
    - Support Vector Machines (SVM) are a method that uses points in a transformed problem space that best separate classes into two groups. Classification for multiple classes is supported by a one-vs-all method. SVM also supports regression by modeling the function with a minimum amount of allowable error

### Cross Validation
- 모델 구축 후 성능 검증을 위해 전체 Dataset을 Train, Validation과 Test로 나눈다. 
- Testset은 최적화된 파라메터로 구축된 최종 모델의 성능을 파악하기 위해 단 1회만 사용한다. 
- 최적화 파라메터는 Scikit-learn에서 제공하는 grid_serach를 이용해 구한다.
- Dataset을 나눌 때 test_size 옵션으로 Train, Test의 비율을 설정할 수 있고, random_state로 seed 값을 지정할 수 있다.
- 데이터 샘플이 너무 많다면, 연상 비용이 크게 증가할 수 있어 샘플링이 필요하다.

```python
# 샘플링 예시 코드 / frac에는 샘플링셋의 비율을 입력, Replace는 비복원으로 지정(False)
df_sampled = df.sample(frac=.1, replace=False) 
```


In [ ]:
from sklearn.cross_validation import train_test_split

# set ind vars and target var
X = df5.drop('group', axis=1)
y = df5.group

# split train, test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# Scaling
scaler = MinMaxScaler()

# fit_transform
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [ ]:
# max
print(X_train_scaled.max(axis=0))
print(X_test_scaled.max(axis=0))
print(' ')
# min
print(X_train_scaled.min(axis=0))
print(X_test_scaled.min(axis=0))

In [ ]:
print(X_train_scaled.shape)
print(y_train.shape)
print(X_test_scaled.shape)
print(y_test.shape)

### 모델 파라메터 설정
- 기본 모델: **Logistic Regression** 
    - http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
- 주요 파라메터 (C)
    - C 값 (기본값 = 1)
    - C 값이 작으면 Penalty 강해짐 (단순 모델)
    - C 값이 크면 Penalty 약해짐 (정규화 없어짐)
    - 보통 로그스케일로 지정(10배씩) = 0.01, 0.1, 1, 10
- penalty
    - L2: Ridge, 일반적으로 사용 (default)
    - L1: LASSO, 변수가 많아서 줄여야할 때 사용, 모델의 단순화 및 해석에 용이

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# set params
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'penalty': ['l1', 'l2']}

# grid search
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5)

# fit
grid_search.fit(X_train_scaled, y_train)

#### How the grid_search module works:
```python

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# SET default
best_score = 0

# iterataion
for r in ['l1', 'l2']:
    for C in [0.001, 0.01, 0.1, 1, 10, 100]:
        lm = LogisticRegression(penalty = r, C=C)
        scores = cross_val_score(lm, X_train, y_train, cv=5)
        score = np.mean(scores)
        if score > best_score:
            best_score = score
            best_parameters = {'C': C, 'penalty': r}
            
```            

In [ ]:
print(grid_search.best_params_)
print(grid_search.best_score_)
print(grid_search.best_estimator_)

In [ ]:
grid_search.score(X_test_scaled, y_test) # accuracy

In [ ]:
grid_search.predict(X_test_scaled)

In [ ]:
print(len(grid_search.predict(X_test_scaled)))
print(len(y_test))

### 1차 모델 평가 (about the first model)

In [ ]:
print('when grid searching: ', grid_search.best_score_)
print('at the trainset:, ', grid_search.score(X_test_scaled, y_test))

In [ ]:
# 실제 테스트셋의 label 분포
y_test.value_counts()

In [ ]:
# 모델 예측 결과
pd.Series(grid_search.predict(X_test_scaled)).value_counts()

In [ ]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(grid_search.predict(X_test), y_test))

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(grid_search.predict(X_test), y_test))

In [ ]:
# ROC plot
from sklearn.metrics import roc_curve, auc

false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, grid_search.predict(X_test))
roc_auc = auc(false_positive_rate, true_positive_rate)

fig = plt.figure(figsize=(6,4))
plt.title('Receiver Operating Characteristic')
plt.plot(false_positive_rate, true_positive_rate, 'b', label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.1,1.2])
plt.ylim([-0.1,1.2])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

### Upsampling & Downsampling for imbalanced data
1. Collect More Data (if possible)
2. Resampling the Dataset
    - oversampling
        - no information loss, perform better than undersampling
        - overfitting issues (because of duplicates)
    - undersampling
        - help improve run time and storage problems
        - information loss, biased dataset
3. Generate Synthetic Samples

In [ ]:
# orginal dataset
df5.group.value_counts()

In [ ]:
df5.group.value_counts().transform(lambda x: x / x.sum())

In [ ]:
def oversampling(df):    

    df_pay_only = df.query("group == 1")
    df_pay_only_over = pd.concat([df_pay_only, df_pay_only, df_pay_only], axis=0) 
    df_over = pd.concat([df, df_pay_only_over], axis=0)

    return df_over

In [ ]:
df5_over = oversampling(df5)

In [ ]:
df5_over.group.value_counts().transform(lambda x: x / x.sum())

In [ ]:
X = df5_over.drop("group", axis=1)
y = df5_over.group

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Scaling
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [ ]:
print(X_train_scaled.shape)
print(y_train.shape)
print(X_test_scaled.shape)
print(y_test.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# set params
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'penalty': ['l1', 'l2']}

# grid search
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5)

# fit
grid_search.fit(X_train_scaled, y_train)

In [ ]:
print(grid_search.best_params_)
print(grid_search.best_score_)
#print(grid_search.best_estimator_)
print(grid_search.score(X_test_scaled, y_test))

In [ ]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(grid_search.predict(X_test_scaled), y_test))

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(grid_search.predict(X_test_scaled), y_test))

In [ ]:
# ROC plot
from sklearn.metrics import roc_curve, auc

false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, grid_search.predict(X_test_scaled))
roc_auc = auc(false_positive_rate, true_positive_rate)

fig = plt.figure(figsize=(6,4))
plt.title('Receiver Operating Characteristic')
plt.plot(false_positive_rate, true_positive_rate, 'b', label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.1,1.2])
plt.ylim([-0.1,1.2])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

### The model performance of the first application
- process
    - oversampling
    - dataset split
    - minmax scale
    - logistic regression, grid search, k-fold(5)
    - evaluation
    - Baseline score: **Precision: 0.5, Recall: 0.76, AUC: 0.74**
- How to improve
    - **There seems no overfitting issues**
        - how to avoid overfitting: collect more data
            - regularization
            - feature deduction
            - collect more samples
    - **Feature Engineering**
        - Other Scaling and Transformation
        - Feature selection or creation
            - Polynomial / Interactions
            - new features
        - Transformation
            - log, exp, sqrt (if not tree-based model)
            - Numeric to Categorical
    - **Model, Parameter Tuning**
        - KNN
        - NB
        - SVM
        - RF
        - NN .. any classification models


### How to Improve
- scale
- distribution transformation
- feature selection/polynomial or interation
- apply other models

### Change Scale to z-score & pipeline

In [ ]:
df5_over.head() # after removes outliers

In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:
from sklearn.pipeline import Pipeline

def pipeline_logit(X_train, y_train):

    scaler = StandardScaler()
    logit_model = LogisticRegression()

    pipe = Pipeline([('scaler', scaler), ('model', logit_model)])

    param_grid = [{'model__C': [0.001, 0.01, 0.1, 1, 10, 100],
                  'model__penalty': ['l1', 'l2']}]

    grid_search = GridSearchCV(pipe, param_grid, cv=5)
    grid_search.fit(X_train, y_train)
    
    return grid_search

In [ ]:
grid_search = pipeline_logit(X_train, y_train)

In [ ]:
print("best score: ", grid_search.best_score_)
print("best score: ", grid_search.best_params_)

In [ ]:
print(classification_report(grid_search.predict(X_test), y_test))

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, grid_search.predict(X_test))
roc_auc = auc(fpr, tpr)

print(roc_auc)

### Transfrom Distribution

In [ ]:
df5_over.describe()

In [ ]:
df5_over.hist(bins=15, color='darkblue', figsize=(18,14), grid=False);

In [ ]:
df5_over_log = df5_over.loc[:,:'traffic'].apply(lambda x: np.log(x + 1)).join(df5_over['group'])

In [ ]:
df5_over_log.describe()

In [ ]:
df5_over_log.hist(bins=15, color='brown', figsize=(18,14), grid=False);

In [ ]:
X = df5_over_log.drop("group", axis=1)
y = df5_over_log.group

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
grid_search = pipeline_logit(X_train, y_train)

In [ ]:
def evaluation(grid, X_test, y_test):
    
    print(classification_report(grid.predict(X_test), y_test))

    print("best score: ", grid.best_score_)
    print("best params: ", grid.best_params_)

    fpr, tpr, thresholds = roc_curve(y_test, grid.predict(X_test))
    roc_auc = auc(fpr, tpr)
    
    return roc_auc

In [ ]:
evaluation(grid_search, X_test, y_test)

In [ ]:
# ROC plot
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, grid_search.predict(X_test))
roc_auc = auc(false_positive_rate, true_positive_rate)

fig = plt.figure(figsize=(6,4))
plt.title('Receiver Operating Characteristic')
plt.plot(false_positive_rate, true_positive_rate, 'b', label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.1,1.2])
plt.ylim([-0.1,1.2])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

#### The current score: **Precision: 0.77, Recall: 0.88, AUC: 0.86**

### [Feature Selection](https://machinelearningmastery.com/feature-selection-machine-learning-python/)
 - Efficiency
 - [Multicollinearity](https://ko.wikipedia.org/wiki/%EB%8B%A4%EC%A4%91%EA%B3%B5%EC%84%A0%EC%84%B1)
 - How to select
    - Univariate Selection: T-test, ANOVA, Coefficient 
    - Feature Importance (Tree-based model)
    - RFE

In [ ]:
print(len(X_train.columns))
print(len(X_test.columns))

### Univariate Selection
- [F value](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_selection)
- http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest
- 그룹내 분산이 작고, 그룹간 분산이 클 경우 F value가 커짐 (F value가 크다는 의미는 그룹간 통계적 차이가 크다는 것을 의미)

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

def pipeline_logit_kbest(X_train, y_train):

    select = SelectKBest(score_func=f_classif) # if regression problem, score_func=f_regression

    scaler = StandardScaler()
    logit_model = LogisticRegression()

    pipe = Pipeline([('scaler', scaler), ('feature_selection', select), ('model', logit_model)])

    param_grid = [{'feature_selection__k': [3,5,7],
                  'model__C': [0.001, 0.01, 0.1, 1, 10, 100],
                  'model__penalty': ['l1', 'l2']
                  }]

    grid_search = GridSearchCV(pipe, param_grid, cv=5)
    grid_search.fit(X_train, y_train)
    
    return grid_search

In [ ]:
grid_search_kbest = pipeline_logit_kbest(X_train, y_train)

In [ ]:
evaluation(grid_search_kbest, X_test, y_test)

In [ ]:
mask = grid_search_kbest.best_estimator_.named_steps['feature_selection'].get_support()
features_list = list(X_train.columns.values)

selected_features = []
for bool, features in zip(mask, features_list):
    if bool:
        selected_features.append(features)
        
print(selected_features)        

### Feature Importance of ExtraTreesClassifier

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

def pipeline_tree_kbest(X_train, y_train):

    select = SelectKBest(score_func=f_classif) # if regression problem, score_func=f_regression

#    scaler = StandardScaler()
    extra_tree_model = ExtraTreesClassifier()

    pipe = Pipeline([('feature_selection', select), ('model', extra_tree_model)])

    param_grid = [{'feature_selection__k': [5,7],
                   'model__max_depth': [4, 6], # max_depth: The maximum depth of the tree.
                   'model__n_estimators': [10, 50], # n_estimators: The number of trees in the forest.
                   'model__min_samples_split': [50, 100]}]

    grid_search = GridSearchCV(pipe, param_grid, cv=3)
    grid_search.fit(X_train, y_train)
    
    return grid_search

In [ ]:
grid_search_tree = pipeline_tree_kbest(X_train, y_train)

In [ ]:
evaluation(grid_search_tree, X_test, y_test)

In [ ]:
mask = grid_search_tree.best_estimator_.named_steps['feature_selection'].get_support()
feature_importance = grid_search_tree.best_estimator_.named_steps['model'].feature_importances_

features_list = list(X_train.columns.values)

selected_features = []
for bool, features in zip(mask, features_list):
    if bool:
        selected_features.append(features)

# create a df        
feature_importance_pd = pd.DataFrame(list(zip(selected_features, feature_importance)),\
                                    columns=['features', 'importance'])\
                          .set_index("features").sort_values("importance")

# visiual
feature_importance_pd.plot(kind='barh', color='darkblue')

plt.title("Feature Importance")
plt.grid(color='lightgrey', alpha=0.5, linestyle='--')
plt.tight_layout()

### Q)  RFE (recursive feature elimination)
- Backward 방식중 하나로 모든 변수를 다 포함시키고 반복해서 학습을 하면서 중요하지 않은 변수를 하나씩 제거하는 방식
    - [API DOC](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html#sklearn.feature_selection.RFE)
- 위 방식을 이용해서 원하는 모델을 이용해 Feature Selection(elimination)을 해보세요. 

### Q) Random Forest, SVM이나 NB, Neural Network 등 다른 모델도 파이프라인에 사용해보세요.
```python
# KNN
from sklearn.neighbors import KNeighborsClassifier

params_grid = [{'n_neighbors': [3, 5, 10], # default: 5
                'metric': ['euclidean', 'manhattan']
                # cityblock’, ‘cosine’, ‘euclidean’, ‘l1’, ‘l2’, ‘manhattan’
               }]

# SVC
from sklearn.svm import SVC

params_grid = [{'C': [1, 10], # Penalty parameter C of the error term
                'gamma': [1, 10] # Higher the value of gamma, will try to exact fit
                'kernel': ['linear', 'rbf']
               }]

# neural_network
from sklearn.neural_network import MLPClassifier

params_grid = [{'solver': [1, 10],
                'hidden_layer_sizes': [(5,2), (3,3)]
               }]

```